In [60]:
#import libraries
import requests
import pandas as pd
from datetime import datetime
import os
import html

In [62]:
#Clean gas and meteo

In [64]:
#download dataset
df_lines = pd.read_csv('./Online_Data/referentiel-des-lignes.csv', sep=';')
stops_data = pd.read_csv('./Online_Data/arrets.csv', sep=';')
delays_metro = pd.read_csv("./Collected_Data/metro_delays.csv")
delays_rer = pd.read_csv("./Collected_Data/rail_delays.csv")
onTime_metro = pd.read_csv("./Collected_Data/metro_onTime.csv")
onTime_rer = pd.read_csv("./Collected_Data/rail_onTime.csv")
metro_incident = pd.read_csv("./Collected_Data/metro_line_reports.csv")
rer_incident = pd.read_csv("./Collected_Data/rer_line_reports.csv")
trafic2023_ratio = pd.read_csv("./Online_Data/validations-1er-semestre.csv", sep=';')
trafic2023_raw = pd.read_csv("./Online_Data/validations-reseau.csv", sep=";")
#gas = pd.read_csv("./Online_Data/prix-des-carburants-en-france-flux-instantane-v2.csv", sep=";") #needs cleaning/filtering

In [65]:
#create filter
stops_filter = [22086, 463013, 22136, 462993, 21964, 462969, 22125, 463113, 41295, 473921, 473993, 41354, 474060, 474061]
name_filter = ["CH.D.G.ETOILE", "CHATELET", "SAINT-LAZARE","ST-GERM.D.PRES", "BLANCHE","AVENUE DU PRESIDENT KENNEDY","BUNO GIRONVILLE","MASSY PALAISEAU"]
lines_filter = ["C01371", "C01372", "C01374" ," C01382", "C01742", "C01743", "C01727", "C0172"]

In [66]:
#clean dataset
line_refs = df_lines[(~df_lines['TransportSubmode'].isin(['suburbanRailway', 'regionalRail', 'railShuttle']))]
line_refs = line_refs[['ID_Line', 'TransportMode', 'Name_Line']]
line_refs = line_refs.sort_values(by='ID_Line')

stops_data = stops_data[stops_data['ArRType'].isin(['metro', 'rail'])]
stops_data = stops_data.sort_values(by=['ArRType', 'ArRId'])
stops_data = stops_data[['ArRId', 'ArRName', 'ArRType', 'ArRTown']]

metro_incident['ref'] = metro_incident['ref'].str.replace('stop_point:IDFM:', '', regex=False)
metro_incident['ref'] = metro_incident['ref'].str.replace('line:IDFM:', '', regex=False)
metro_incident['ref'] = metro_incident['ref'].str.replace('stop_area:IDFM:', '', regex=False)
metro_incident['message_text'] = metro_incident['message_text'].str.replace('<p>', '', regex=False)
metro_incident['message_text'] = metro_incident['message_text'].str.split('</p>').str[0]
metro_incident['message_text'] = metro_incident['message_text'].str.replace('<br>', '', regex=False) 
metro_incident = metro_incident[metro_incident["channel_name"].isin(["moteur"])]
metro_incident.drop_duplicates(subset=["disruption_id"], inplace=True)
metro_incident.drop(['tags', 'category', 'updated_at', 'channel_name'], axis=1, inplace=True) 

rer_incident['ref'] = rer_incident['ref'].str.replace('stop_point:IDFM:', '', regex=False)
rer_incident['ref'] = rer_incident['ref'].str.replace('line:IDFM:', '', regex=False)
rer_incident['ref'] = rer_incident['ref'].str.replace('stop_area:IDFM:', '', regex=False)
rer_incident['message_text'] = rer_incident['message_text'].str.replace('<p>', '', regex=False)
rer_incident['message_text'] = rer_incident['message_text'].str.replace('</p>', '', regex=False)
rer_incident['message_text'] = rer_incident['message_text'].str.replace('<br>', '', regex=False) 
rer_incident['message_text'] = rer_incident['message_text'].apply(html.unescape)
rer_incident = rer_incident[rer_incident["channel_name"].isin(["moteur"])]
rer_incident.drop_duplicates(subset=["disruption_id"], inplace=True)
rer_incident.drop(['tags', 'category', 'updated_at', 'channel_name'], axis=1, inplace=True) 

delays_metro.drop(['scheduled_arrival','scheduled_departure','arrival_difference','departure_difference'], axis=1, inplace=True)
delays_metro['line_ref'] = delays_metro['line_ref'].str.replace('STIF:Line::', '', regex=False).str.rstrip(':')
onTime_metro.drop(['scheduled_arrival','scheduled_departure','arrival_difference','departure_difference'], axis=1, inplace=True)
onTime_metro['line_ref'] = delays_metro['line_ref'].str.replace('STIF:Line::', '', regex=False).str.rstrip(':')

mapping = dict(zip(name_filter, stops_filter))
trafic2023_ratio['LIBELLE_ARRET_REA'] = trafic2023_ratio['LIBELLE_ARRET'].replace(mapping)
trafic2023_ratio.drop(["lda"],axis=1, inplace=True)

trafic2023_raw['LIBELLE_ARRET_REA'] = trafic2023_raw['LIBELLE_ARRET'].replace(mapping)
trafic2023_raw.drop(["lda"],axis=1, inplace=True)

In [69]:
#filter dataset
stops_data = stops_data[stops_data['ArRId'].isin(stops_filter)]

metro_incident = metro_incident[
    metro_incident['ref'].isin(map(str, stops_filter + lines_filter))
]

rer_incident = rer_incident[
    rer_incident['ref'].isin(map(str, stops_filter + lines_filter))
]

trafic2023_ratio = trafic2023_ratio[trafic2023_ratio['LIBELLE_ARRET_REA'].isin(stops_filter)]

trafic2023_raw = trafic2023_raw[trafic2023_raw['LIBELLE_ARRET_REA'].isin(stops_filter)]

In [78]:
#merge dataset
merged_metro = pd.concat([delays_metro, onTime_metro], ignore_index=True)
merged_metro = pd.merge(merged_metro, stops_data, left_on='stop_reference', right_on='ArRId')
merged_metro.drop(['ArRId', 'ArRName', 'ArRType', 'transport_mode', 'recorded_at_time'], axis=1, inplace=True) 
merged_metro = merged_metro.sort_values(by='real_arrival')

merged_rer = pd.concat([delays_rer, onTime_rer], ignore_index=True)
merged_rer = pd.merge(merged_rer, stops_data, left_on='stop_reference', right_on='ArRId')
merged_rer.drop(['ArRId', 'ArRName', 'ArRType', 'transport_mode', 'recorded_at_time'],  axis=1, inplace=True)
merged_rer = merged_rer.sort_values(by='real_arrival')

In [80]:
#Clean Gas Price data
def clean_gas_data(file_path, fuel_type):
    # Load CSV file
    df = pd.read_csv(file_path, skiprows=3, sep=';')  # Skip the first 3 metadata rows
    df.rename(columns={df.columns[1]: f'{fuel_type}'}, inplace=True)
    df = df.iloc[:, :-1]
    return df
    
gas_95_df = clean_gas_data("./Online_Data/Gas/octane_95.csv", "95")
gas_98_df = clean_gas_data("./Online_Data/Gas/octane_98.csv", "98")
gas_e10_df = clean_gas_data("./Online_Data/Gas/95-E10.csv", "E10")
gazole_df = clean_gas_data("./Online_Data/Gas/gazole.csv", "gazole")

merged_metro['Période'] = pd.to_datetime(merged_metro['timestamp']).dt.strftime('%Y-%m')
merged_metro = merged_metro.merge(gas_95_df, on='Période', how='left')
merged_metro = merged_metro.merge(gas_98_df, on='Période', how='left')
merged_metro = merged_metro.merge(gas_e10_df, on='Période', how='left')
merged_metro = merged_metro.merge(gazole_df, on='Période', how='left')

# Drop unnecessary columns
merged_metro.drop(columns=['Période'], inplace=True)
merged_metro

,timestamp,stop_reference,stop_name,line_ref,destination_name,departure_status,real_arrival,real_departure,ArRTown,95,98,E10,gazole
0,2024-10-30 01:20:26,22086,Charles de Gaulle-Etoile,C01371,Château de Vincennes,delayed,2024-10-30T00:28:39.675Z,2024-10-30T00:28:39.675Z,Paris 17e,1.76,1.82,1.72,1.63
1,2024-10-30 02:20:27,22086,Charles de Gaulle-Etoile,C01371,Château de Vincennes,delayed,2024-10-30T01:28:52.695Z,2024-10-30T01:28:52.695Z,Paris 17e,1.76,1.82,1.72,1.63
2,2024-10-30 03:20:24,22086,Charles de Gaulle-Etoile,C01371,Château de Vincennes,delayed,2024-10-30T02:26:05.147Z,2024-10-30T02:26:05.147Z,Paris 17e,1.76,1.82,1.72,1.63
3,2024-10-30 04:20:31,22086,Charles de Gaulle-Etoile,C01371,Château de Vincennes,delayed,2024-10-30T03:29:28.250Z,2024-10-30T03:29:28.250Z,Paris 17e,1.76,1.82,1.72,1.63
4,2024-11-02 22:21:13,463013,Charles de Gaulle-Etoile,C01371,La Défense (Grande Arche),delayed,2024-11-02T21:23:35.241Z,2024-11-02T21:23:35.241Z,Paris 17e,1.76,1.82,1.72,1.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,2024-12-19 18:09:29,21964,Saint-Lazare,C01376,Mairie d'Issy,onTime,2024-12-19T17:18:34.313Z,2024-12-19T17:18:34.313Z,Paris 9e,NaN,NaN,NaN,NaN
169,2024-12-19 18:11:24,463113,Blanche,C01383,Porte Dauphine (Maréchal de Lattre de Tassigny),onTime,2024-12-19T17:18:48.947Z,2024-12-19T17:18:48.947Z,Paris 9e,NaN,NaN,NaN,NaN
170,2024-12-19 18:10:07,22125,Blanche,C01377,Nation,onTime,2024-12-19T17:20:52.506Z,2024-12-19T17:20:52.506Z,Paris 9e,NaN,NaN,NaN,NaN
171,2024-12-19 18:10:07,22125,Blanche,C01378,Nation,onTime,2024-12-19T17:22:01.501Z,2024-12-19T17:22:01.501Z,Paris 9e,NaN,NaN,NaN,NaN


In [ ]:
#

In [170]:
#add gas and meteo

In [160]:
#split by stop
CDG = merged_metro[merged_metro['stop_name'].isin(['Charles de Gaulle-Etoile'])]
SGP = merged_metro[merged_metro['stop_name'].isin(['Saint-Germain des Prés'])]
BL = merged_metro[merged_metro['stop_name'].isin(['Blanche'])]
SL = merged_metro[merged_metro['stop_name'].isin(['Saint-Lazare'])]
APK = merged_rer[merged_rer['stop_name'].isin(['Avenue du Président Kennedy'])]
CLH = merged_rer[merged_rer['stop_name'].isin(['Châtelet - Les Halles'])]
GBG = merged_rer[merged_rer['stop_name'].isin(['Gare de Buno Gironville'])]
MP = merged_rer[merged_rer['stop_name'].isin(['Massy - Palaiseau'])]